In [192]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.optim import RMSprop

import re
import string

In [193]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [194]:
# Read data
with open('mal-eng\mal.txt','r') as file:
    data = file.read()

In [195]:
print(data[:500])

Hello.	നമസ്കാരം.	CC-BY 2.0 (France) Attribution: tatoeba.org #1858850 (LanguageExpert) & #651913 (jjrodz)
Really?	ശരിക്കും?	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #7896041 (lonewolfie)
Help me.	എന്നെ സഹായിക്കൂ.	CC-BY 2.0 (France) Attribution: tatoeba.org #266065 (Zifre) & #780454 (jjrodz)
Welcome.	സ്വാഗതം.	CC-BY 2.0 (France) Attribution: tatoeba.org #138919 (CM) & #7896035 (lonewolfie)
I forgot.	ഞാന്‍ മറന്നു.	CC-BY 2.0 (France) Attribution: tatoeba.org #436603 (lukasz


In [196]:
inp_tar_data = [text.split('\t')[:2] for text in data.split('\n')]

In [197]:
print(inp_tar_data[:5])

[['Hello.', 'നമസ്കാരം.'], ['Really?', 'ശരിക്കും?'], ['Help me.', 'എന്നെ സഹായിക്കൂ.'], ['Welcome.', 'സ്വാഗതം.'], ['I forgot.', 'ഞാന്\u200d മറന്നു.']]


In [198]:
inp_data = []
tar_data = []
for item in inp_tar_data:
    if len(item)<2:
        continue
    inp_data.append(item[0])
    tar_data.append(item[1])

In [199]:
print('number of data in inp: ',len(inp_data))
print('number of data in tar: ',len(tar_data))

number of data in inp:  614
number of data in tar:  614


In [200]:
print(list(zip(inp_data,tar_data[:4])))

[('Hello.', 'നമസ്കാരം.'), ('Really?', 'ശരിക്കും?'), ('Help me.', 'എന്നെ സഹായിക്കൂ.'), ('Welcome.', 'സ്വാഗതം.')]


In [201]:
# Remove puctuations
def remove_puctuations(word):
    word = re.sub('[{}]'.format(string.punctuation),repl='',string=word)
    return word.lower()

In [202]:
inp_data = list(map(remove_puctuations,inp_data))
tar_data = list(map(remove_puctuations,tar_data))

In [203]:
print(tar_data[:4])

['നമസ്കാരം', 'ശരിക്കും', 'എന്നെ സഹായിക്കൂ', 'സ്വാഗതം']


In [204]:
class LangPorcess:
    def __init__(self):

        self.__word2int = {}
        self.__int2word = {}
        self.__vocabsize = 2
        self.__word2int['<pad>'] = 0
        self.__int2word[0] = '<pad>'
        self.__int2word[1] = '<eos>'
        self.__word2int['<eos>'] = 1
        self.__word_freq = {}
        self.__integer_encoded =[]

    def fit(self,X):
        X_split_words = [text.split(' ') for text in X]
        self.read_sentences(X_split_words)
        self.__integer_encoding(X_split_words)
        
        
    def read_sentences(self,X):
        # Loop thourgh each sentence and read words
        for sentence in X:
            for word in sentence:
                if word not in self.__word2int.keys():
                    self.__word_freq[word] = 1
                    self.read_words(word)
                else:
                    self.__word_freq[word] += 1


    
    def read_words(self,word):
            self.__word2int[word] = self.__vocabsize
            self.__int2word[self.__vocabsize] = word
            self.__vocabsize+=1

    def get_word_frequency(self):
        return dict(sorted(self.__word_freq.items(),
                           key=lambda x:x[1],
                           reverse=True))
    
    def get_word2index(self):
        return self.__int2word
    
    def get_index2word(self):
        return self.__word2int
    
    def __integer_encoding(self,X):  
        for word in X:
            torch_tensor = torch.tensor(list(map(lambda word : self.__word2int[word],word)))
            self.__integer_encoded.append(torch_tensor)

    def get_integer_encoding(self,padding=True,max_len=None):
        if padding:
            return self.__pad_sequence(self.__integer_encoded,max_len)
        else:
            return self.__integer_encoded
        
    def __pad_sequence(self,X,max_len):
            maxlen_padded = pad_sequence(self.__integer_encoded,padding_value=0,batch_first=True)
            if max_len:
                return maxlen_padded[:,:max_len]
            else:
                return maxlen_padded
    
    def get_vocabsize(self):
        return self.__vocabsize
       
        

In [205]:
inp_lang = LangPorcess()
inp_lang.fit(inp_data)

tar_lang = LangPorcess()
tar_lang.fit(tar_data)

In [206]:
inp_integers = inp_lang.get_integer_encoding(max_len=11)
tar_integers = tar_lang.get_integer_encoding(max_len=7)

In [207]:
class TorchDataset(Dataset):
    def __init__(self,enc_sentance,dec_sentance):
        super().__init__()
        self.enc_sentance = enc_sentance
        self.dec_sentance = dec_sentance

    def __len__(self):
        return len(self.enc_sentance)

    def __getitem__(self, index):
        return self.enc_sentance[index], self.dec_sentance[index]



In [208]:
data = TorchDataset(enc_sentance=inp_integers,dec_sentance=tar_integers)

In [209]:
dataloader = DataLoader(data,
                        batch_size=32,
                        shuffle=True,
                        pin_memory=True)

In [210]:
class Encoder(nn.Module):
  def __init__(self, 
               embedding_size, 
               hidden_size, 
               vocab_size):
    super().__init__()

    # encoder initialization
    self.embedding = nn.Embedding(num_embeddings=vocab_size,
                             embedding_dim=embedding_size)
    self.encoder_lstm = nn.LSTM(input_size=embedding_size,
                           hidden_size=hidden_size,
                           batch_first=True)

  def forward(self,input):
    # network flow
    embedding_input = self.embedding(input)
    encoder_outputs, (final_hidden_state, final_cell_state) = self.encoder_lstm(embedding_input)
    
    return encoder_outputs.to(device), final_hidden_state.to(device), final_cell_state.to(device)




In [211]:
class Attention(nn.Module):
  def __init__(self, hidden_size):
    super().__init__()

    self.network = nn.Sequential(
          nn.Linear(2*hidden_size,hidden_size),
          nn.SELU(),
          nn.Linear(hidden_size,1),
          nn.Softmax(dim=1)
        )
    
  def forward(self,encoder_outputs,hidden_state):
    """ Concat encoeder_output and hidden_state, encoder_output shape = (32,timestept,hidden_size), hidden_state shape = 32,1,hidden_side
    first we need to make it same shape to concat hidden_state should be 32,timestep hidden_size, timestpe will be repeatation of same one vector from hidden size"""

    encoder_timestep_len = encoder_outputs.size(1)  # hidden_size will be (1,32,hidden_size) according to doc we need to change
    hidden_state = hidden_state.permute(1,0,2) # shape: (32,1,5)
    hidden_repeated = hidden_state.repeat(1,encoder_timestep_len,1) # hidden_state repetation 

    # concat with encoder_output and hidden output
    encoder_hidden_concat = torch.concat((encoder_outputs,hidden_repeated),dim=-1) # shape : 32,timestep,hidden_size*2
    weights = self.network(encoder_hidden_concat) # (32,timestepe,1)
    weights = weights.permute(0,2,1) # for bmm (32,1,timesteps)
    context_vectores = weights.bmm(encoder_outputs) # 32,1,hidden_size
    context_vectores = context_vectores.permute(1,0,2) # convert back to the way lstm take hidden state

    return context_vectores

In [212]:
class Decoder(nn.Module):
  def __init__(self,
               embedding_dim,
               hidden_size,
               vocab_size_tr,
               max_len=20,
               sos_token=1,
               ):
    super().__init__()
    
    self.MAX_LEN = max_len
    self.SOS_TOKEN = sos_token

    # Layers Initialization
    self.embedding_layer = nn.Embedding(vocab_size_tr, embedding_dim)
    self.lstm_layer = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
    self.fnn = nn.Linear(hidden_size ,vocab_size_tr)
    self.attention_vector = Attention(hidden_size)

  def forward(self,
              encoder_outputs,
              hidden_state,
              cell_state,
              target_output=None):
    
    batch_size = encoder_outputs.shape[0]     # encoder gets the input from train loader which defines the batchsize
    decoder_input = torch.empty(size=(batch_size,1),dtype=torch.long).fill_(self.SOS_TOKEN).to(device)     # Initialize first input [32 sos_tokens]
    decoder_outputs = []

    for i in range(self.MAX_LEN):
      output_logits ,hidden_state, cell_state = self.forward_step(encoder_outputs, decoder_input, hidden_state, cell_state)
      decoder_outputs.append(output_logits.unsqueeze(1))      # decoder ouput = [(32,vocab_size),...(32,vocab_size)], this list will have max_len item , lastly we will concat this to make (32,max_len,vocab_size)

      # teacher_forcing, occurs if we give target_output in the decoder
      if target_output != None:
        decoder_input = target_output[:,i].unsqueeze(1)
      else:
        _, decoder_input = output_logits.topk(1,dim=-1)

    decoder_final_output = torch.cat(decoder_outputs,dim=1)
    return decoder_final_output

  def forward_step(self,encoder_outputs, decoder_input, hidden_state, cell_state):
    embedded_decoder_input = self.embedding_layer(decoder_input)      # embedded shape : (32,1,embedd_size), here 1 , becuase we are giving each word or token to decoder and make it predict next word
    lstm_output, (decoder_hidden, decoder_cell) = self.lstm_layer(embedded_decoder_input, (hidden_state, cell_state))     # lstm_output: (32,1,hidden_size)
    output_logit = self.fnn(lstm_output.squeeze(1))   # squeeze (32,1,hidden_size) -> 32,hidden_size
    hidden_state = self.attention_vector(encoder_outputs, decoder_hidden)     # ouput_logits: (32,vocab_size) , 32 prediction of word , we will pic top item

    return output_logit ,hidden_state , decoder_cell

In [213]:
class Seq2SeqAttentionModel(nn.Module):
  def __init__(self,
               embedding_size,
               hidden_size,
               vocab_size_en,
               vocab_size_tr,
               max_len=10):
    super().__init__()

    self.encoder = Encoder(embedding_size,
                           hidden_size,
                           vocab_size_en).to(device)

    self.decoder = Decoder(embedding_size,
                           hidden_size,
                           vocab_size_tr,
                           max_len,
                           sos_token=1).to(device)

  def forward(self, input, target_output):
    encoder_outputs = self.encoder(input)
    decoder_output = self.decoder(*encoder_outputs, target_output)

    return decoder_output

In [214]:
def train(train_dataloader, 
          inp_vocabsize,
          tar_vocabsize, 
          embedding_size=128,
          hidden_size=128,
          epoch=10,
          max_len=7):

    model = Seq2SeqAttentionModel(embedding_size,
                                hidden_size,
                                inp_vocabsize,
                                tar_vocabsize,
                                max_len=max_len)
    model.to(device)
    criteria = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = RMSprop(model.parameters())


    for epoch_num in range(epoch):
        total_loss = 0
        for input_sentance, target_sentance in train_dataloader:
            input_sentance,  target_sentance = input_sentance.to(device), target_sentance.to(device)
            output = model(input_sentance,target_sentance)
            output.to(device)
            target_sentance = target_sentance.view(-1)
            output = output.view(target_sentance.shape[0],-1)
            optimizer.zero_grad()
            loss = criteria(output,target_sentance)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f'loss in {epoch_num} epoch:',total_loss)
    
    return model



In [215]:
embedding_size = 128
hidden_size = 128
inp_vocabsize = inp_lang.get_vocabsize() + 1
tar_vocabsize = tar_lang.get_vocabsize() + 1
epoch = 100
max_len = 7

model= train(dataloader,
             inp_vocabsize,
             tar_vocabsize,
             epoch=10,
             max_len=max_len,
             embedding_size=embedding_size,
             hidden_size=hidden_size)

loss in 0 epoch: 141.6688199043274
loss in 1 epoch: 112.01326656341553
loss in 2 epoch: 75.16450595855713
loss in 3 epoch: 44.99920880794525
loss in 4 epoch: 33.40816533565521
loss in 5 epoch: 28.004756331443787
loss in 6 epoch: 24.32692378759384
loss in 7 epoch: 21.8040491938591
loss in 8 epoch: 19.43064421415329
loss in 9 epoch: 17.461343705654144


In [216]:
def evaluate(model, dataloader):
    model.to(device)
    criteria = nn.CrossEntropyLoss(ignore_index=0)
    val_loss = 0
    model.eval()

    with torch.no_grad():
        for input_sentance, target_sentance in dataloader:
            input_sentance,  target_sentance = input_sentance.to(device), target_sentance.to(device)
            output = model(input_sentance,target_sentance)
            target_sentance = target_sentance.view(-1)
            output = output.view(target_sentance.shape[0],-1)
            loss = criteria(output,target_sentance)
            val_loss += loss.item()

    print(f'Evaluation loss:',val_loss)





In [217]:
evaluate(model,dataloader)

Evaluation loss: 16.058470845222473
